# Using NCrystal as a backend in full-fledged Monte Carlo simulation applications

In this notebook we discuss how the full potential of NCrystal is realised when used as a physics engine for applications which track neutrons as they traverse complicated geometries and undergo multiple scatterings in complex geometries.  In particular we will mention how NCrystal is used in applications like McStas, OpenMC, or Geant4.

## Preamble
Install dependencies and prepare plots. Feel free to edit as you wish:

In [1]:
#By default we only do pip installs on Google Colab, but you
#can set the variable in the next line to True if you need it:
always_do_pip_installs = False
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    from importlib.util import find_spec as _fs
    if not _fs('NCrystal'):
        %pip -q install ncrystal[plot] ipympl
#enable inline and interactive matplotlib plots:
if google_colab:
    google_colab.output.enable_custom_widget_manager()
    %matplotlib inline
else:
    %matplotlib ipympl
import matplotlib
matplotlib.rcParams.update({"figure.autolayout": True})
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  4000000, "too old NCrystal found"
NC.test() #< quick unit test that installation works

Tests completed succesfully


## Using NCrystal with other codes.
Apart from being used by itself, the original reason for developing NCrystal was indeed to serve as a backend engine for existing Monte Carlo codes, providing a consistent high-fidelity neutron scattering physics to those. One advantage here is of course that it allows both developers and users across all of the supported codes to collaborate on the neutron scattering models. In this notebook, we will focus on three such applications that are all well-supported, widely used, and open-source: OpenMC, McStas, and Geant4.

### OpenMC
Using NCrystal materials in openmc is supported since OpenMC release 13.3, and uses a nice simple syntax in the Python API (see also https://docs.openmc.org/en/stable/usersguide/materials.html#adding-ncrystal-materials):

```python
mat = openmc.Material.from_ncrystal('Polyethylene_CH2.ncmat;temp=50C')
```

which when used in a complete OpenMC project, results in the following material entry being added to the `materials.xml` produced:

```xml
  <material cfg="Polyethylene_CH2.ncmat;temp=50C" id="1" temperature="323.15">
    <density units="g/cm3" value="0.92" />
    <nuclide ao="0.66656284" name="H1" />
    <nuclide ao="0.00010382666666666666" name="H2" />
    <nuclide ao="0.32964066666666664" name="C12" />
    <nuclide ao="0.003692666666666666" name="C13" />
  </material>
```

Temperature, density and material composition were all created automatically from the cfg-string, and the cfg-string itself was also encoded. Upon launching the simulation with the OpenMC binary executable `openmc`, it will handle the material as usual, except that low-energy neutron scattering physics (currently defined as ($E<5eV$) will be provided by the algorithms in NCrystal.

A few issues might warrent attention:

1. If you try to assemble the above xml manually, it is rather unlikely that you will get the base densities and compositions right. It is safest to stick to let the Python API compose the xml for you.
2. After creation with `mat=openmc.Material.from_ncrystal(..)`, you can not use the usual OpenMC API to modify the material density, temperature, or composition. So be sure to reflect the final desired material inside the NCrystal cfg-string.
3. The OpenMC binaries must have been built with NCrystal support, or your job will fail once you launch the simulation (you can check for this by running the command `openmc -v`). Specifically (as documented on https://docs.openmc.org/en/stable/usersguide/install.html) you must supply the CMake flag
`cmake -DOPENMC_USE_NCRYSTAL=on ..` (and make sure NCrystal is available already).
Note: we have agreement from OpenMC developers to enable NCrystal support by default in the conda-forge version of OpenMC. So in "the near future" (summer/fall 2023) conda users will always have NCrystal support available in OpenMC.

For reference, here is a complete small example of an OpenMC project in Python using NCrystal.

```python
    import openmc
    # Materials
    openmc_mat = openmc.Material.from_ncrystal('Polyethylene_CH2.ncmat;temp=50C')
    # Geometry
    s1 = openmc.Sphere(r=10, boundary_type='vacuum')
    c1 = openmc.Cell(region=-s1, fill=openmc_mat)
    geometry = openmc.Geometry([c1])
    # Execution settings
    settings = openmc.Settings()
    settings.source = openmc.Source(energy=openmc.stats.Discrete(x=[10.0], p=[1.0]))
    settings.run_mode = 'fixed source'
    settings.batches = 10
    settings.particles = 10000
    # Write xml files
    model = openmc.model.Model(geometry=geometry, settings=settings)
    model.export_to_xml()
    #Check resulting materials.xml (we could of course actually RUN OpenMC here instead)
    import pathlib
    print(pathlib.Path('materials.xml').read_text())
```

Credits: The OpenMC-NCrystal bindings originated with the NCrystal developers (esp. J.I. Marquez Damian), but were facilitated along the way by useful technical assistance and a welcoming attitude from the OpenMC developers (esp. Paul Romano).

### McStas (incl. Union and McStasScript)

You can use NCrystal in two ways in McStas. You can either use it for advanced studies with the McStas Union sub-system through the `NCrystal_process` component, or it can be used via the dedicated (and less feature-rich) `NCrystal_sample.comp`.

It is beyond the scope for this notebook to provide a full documentation of McStas, or the Union sub-system, but if you are using McStasScript to compose your instruments, you can add NCrystal materials into your Union geometry using code like:
```python
  import mcstasscript.tools.ncrystal_union as ncunion
  ncunion.add_ncrystal_union_material(instr,
                                      name="myAl",
                                      cfgstr="Al_sg225.ncmat;temp=10C")
  #... usual mcstasscript code for creating volume "myvol" here...
  myvol.set_parameters(radius=0.01,
                       yheight=0.01,
                       material_string='"myAl"',
                       priority=1)
```
This creates the material and gives it the name `"myAl"`, which you must later attach to a particular Union volume, like for instance:
```python
  myvol.set_parameters(radius=0.01, yheight=0.01, material_string='"myAl"', priority=1)
```
If you are instead hand-editing your instrument files, you can generate code which defines Union materials from an NCrystal cfg-string by invoking:

In [2]:
!python3 -mNCrystal.mcstasutils --union myAl 'Al_sg225.ncmat;temp=250K'


/*
   The following code was auto generated by NCrystal v3.9.4 via Python:

     NCrystal.mcstasutils.cfgstr_2_union_instrument_code(
         cfgstr = 'Al_sg225.ncmat;temp=250K',
         name = 'myAl' )

   Please rerun in case of major changes to input data or NCrystal.
*/

COMPONENT myAl_ncrystal_proc = NCrystal_process(
    cfg = "Al_sg225.ncmat;temp=250" )
AT (0,0,0) ABSOLUTE

COMPONENT myAl = Union_make_material(
    process_string = "myAl_ncrystal_proc",
    my_absorption = 1.39136803716641 )
AT (0,0,0) ABSOLUTE

/* End of auto generated code from NCrystal v3.9.4. */



Or you can add `--split` to investigate by physics type:

In [3]:
!python3 -mNCrystal.mcstasutils --union myAl 'Al_sg225.ncmat;temp=250K' --split


/*
   The following code was auto generated by NCrystal v3.9.4 via Python:

     NCrystal.mcstasutils.cfgstr_2_union_instrument_code(
         cfgstr = 'Al_sg225.ncmat;temp=250K',
         name = 'myAl',
         split_by_physics = True )

   Please rerun in case of major changes to input data or NCrystal.
*/

COMPONENT myAl_ncrystal_cohelas_proc = NCrystal_process(
    cfg = "Al_sg225.ncmat;incoh_elas=0;inelas=0;sans=0;temp=250" )
AT (0,0,0) ABSOLUTE

COMPONENT myAl_ncrystal_incohelas_proc = NCrystal_process(
    cfg = "Al_sg225.ncmat;coh_elas=0;inelas=0;sans=0;temp=250" )
AT (0,0,0) ABSOLUTE

COMPONENT myAl_ncrystal_inelas_proc = NCrystal_process(
    cfg = "Al_sg225.ncmat;coh_elas=0;incoh_elas=0;sans=0;temp=250" )
AT (0,0,0) ABSOLUTE

COMPONENT myAl = Union_make_material(
    process_string = "myAl_ncrystal_cohelas_proc,myAl_ncrystal_incohelas_proc,myAl_ncrystal_inelas_proc",
    my_absorption = 1.39136803716641 )
AT (0,0,0) ABSOLUTE

/* End of auto generated code from NCrystal v3.

It should be noted that McStas has a `SHELL` syntax which can also be used to faciliate this invocation from with a classic `.instr` file. Its usage can bee seen in the following example

```c
DEFINE INSTRUMENT example2()
SHELL "python3 -mNCrystal.mcstasutils --union --split myMaterial_Al 'phases<0.01*void.ncmat&0.99*Al_sg225.ncmat;temp=200K>' > materials_snippet.instr"
TRACE
COMPONENT init = Union_init()
AT (0,0,0) ABSOLUTE
%include "materials_snippet.instr"
COMPONENT origin = Progress_bar()
  AT (0, 0, 0) RELATIVE ABSOLUTE
COMPONENT source =   Source_div(lambda0=1.539739, dlambda=0.01, xwidth=0.001, yheight=0.001, focus_aw=1, focus_ah=1)
  AT (0, 0, 0.3) RELATIVE origin
COMPONENT powder_sample = Union_cylinder(yheight=0.01, radius=0.01, priority=1, material_string="myMaterial_Al")
AT (0, 0, 1) RELATIVE origin
COMPONENT master2 = Union_master()
AT (0, 0, 0) RELATIVE powder_sample
COMPONENT powder_pattern_detc = Monitor_nD(
    options = "banana, angle limits=[10 170], bins=500",
    radius = 0.05, yheight = 0.1)
  AT (0, 0, 0) RELATIVE powder_sample
COMPONENT stop = Union_stop()
AT (0,0,0) ABSOLUTE
END
```

On the other hand, the dedicated NCrystal_sample.comp component, embeds NCrystal material simulations into simple shapes (currently boxes, cylinders and spheres), and can be used for components representing samples, filters or monochromators, entrance windows, etc. The component is since McStas v3.3 part of the McStas release itself, and can be used in a .instr file - for instance if you wish to set up an 1cm radius 5cm long cylinder with powdered sapphire you would write:
```c
COMPONENT mysample = NCrystal_sample(cfg="Al2O3_sg167_Corundum.ncmat",
                                     radius=0.01, yheight=0.05)
AT (0, 0, 0) RELATIVE PREVIOUS
```
For more documentation about the NCrystal_sample component, run:
```bash
$> mcdoc NCrystal_sample
```
Or consult the documentation online at https://www.mcstas.org/download/components/

*Credits: The integration into McStas, Union, and McStasScript is largely a result of efforts by Peter Willendrup and Mads Bertelsen working in close collaboration with the NCrystal developers.*

### Geant4

Geant4-bindings allow NCrystal cfg-strings to be used directly in Geant4 material creation code, through the usage of specific helper functions:

```c++
//Include the relevant header:
#include "G4NCrystal/G4NCrystal.hh"

//...

//Create materials directly from cfg-strings:
G4Material * mat_aluminium = G4NCrystal::createMaterial("Al_sg225.ncmat");

//...

//Currently the NCrystal process must then be injected in the following way:
g4runManager->Initialize();
G4NCrystal::installOnDemand();
g4runManager->BeamOn(1000);
```
The last part about injecting a specific NCrystal process is a bit of a technical workaround, and the NCrystal developers are in contact with the Geant4 Hadronic working group (in particular thanks to Alberto Ribbon) about a better solution. But it works for now in non-multithreaded Geant4.

For a fully fledged example, see:

https://github.com/mctools/ncrystal/blob/master/examples/ncrystal_example_g4sim.cc

### Others

While OpenMC, Geant4, and McStas are currently the best known and supported frameworks, other emerging applications are currently integrated with NCrystal, including the Cinema framework from CSNS (Xiao Xiao Cai, et. al.), [ANTS2](https://github.com/andrmor/ANTS2) from LIP-Coimbra (Andrei Morozov, et. al.), and [TOUCANS](https://doi.org/10.1016/j.nima.2023.168190) from CEA (Loïc Thulliez, et. al.).

It is also possibly to benefit partly from NCrystal by using several of the conversion mechanisms we have for creating data-files for other codes. In all cases though, a lot of the physics is lost in the conversion so if at all possible, you should try to use NCrystal directly:
* You can use the `ncrystal_ncmat2hkl` commandline tool to create `.lau` and `.laz` files for various non-NCrystal Bragg diffraction components in McStas.
* You can use the NJOY-NCrystal project to create ENDF files from NCrystal data files, which can then be used in traditional codes like MCNP, etc. For more information see:
  * https://github.com/highness-eu/NJOY-NCrystal-Library
  * K. Ramic, et. al.. "NJOY+NCrystal: an open-source tool for creating thermal neutron scattering libraries"  Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment 1027 (2022) 166227, https://doi.org/10.1016/j.nima.2021.166227
  * J.I. Márquez Damián is working on a more convenient utility which would be able to write ENDF files from NCrystal without needing NJOY (see https://github.com/mctools/ncrystal/issues/183).